# Exercise 2 - Parallel Data Processing with Task Dependencies

**GOAL:** The goal of this exercise is to show how to pass object IDs into remote functions to encode dependencies between tasks.

In this exercise, we construct a sequence of tasks each of which depends on the previous mimicking a data parallel application. Within each sequence, tasks are executed serially, but multiple sequences can be executed in parallel.

In this exercise, you will use Ray to parallelize the computation below and speed it up.

### Concept for this Exercise - Task Dependencies

Suppose we have two remote functions defined as follows.

```python
@ray.remote
def f(x):
    return x
```

Arguments can be passed into remote functions as usual.

```python
>>> x1_id = f.remote(1)
>>> ray.get(x1_id)
1

>>> x2_id = f.remote([1, 2, 3])
>>> ray.get(x2_id)
[1, 2, 3]
```

**Object IDs** can also be passed into remote functions. When the function actually gets executed, **the argument will be a retrieved as a regular Python object**.

```python
>>> y1_id = f.remote(x1_id)
>>> ray.get(y1_id)
1

>>> y2_id = f.remote(x2_id)
>>> ray.get(y2_id)
[1, 2, 3]
```

So when implementing a remote function, the function should expect a regular Python object regardless of whether the caller passes in a regular Python object or an object ID.

**Task dependencies affect scheduling.** In the example above, the task that creates `y1_id` depends on the task that creates `x1_id`. This has the following implications.

- The second task will not be executed until the first task has finished executing.
- If the two tasks are scheduled on different machines, the output of the first task (the value corresponding to `x1_id`) will be copied over the network to the machine where the second task is scheduled.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import time

In [2]:
ray.init(num_cpus=4, include_webui=False, ignore_reinit_error=True)

2021-02-01 17:27:12,828	INFO resource_spec.py:205 -- Starting Ray with 1.07 GiB memory available for workers and up to 0.55 GiB for objects. You can adjust these settings with ray.remote(memory=<bytes>, object_store_memory=<bytes>).
2021-02-01 17:27:13,131	WARNING services.py:1237 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-size' to 'docker run'.


{'node_ip_address': '10.0.10.120',
 'redis_address': '10.0.10.120:46073',
 'object_store_address': '/tmp/ray/session_2021-02-01_17-27-12_827271_601/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-01_17-27-12_827271_601/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2021-02-01_17-27-12_827271_601'}

These are some helper functions that mimic an example pattern of a data parallel application.

**EXERCISE:** You will need to turn all of these functions into remote functions. When you turn these functions into remote function, you do not have to worry about whether the caller passes in an object ID or a regular object. In both cases, the arguments will be regular objects when the function executes. This means that even if you pass in an object ID, you **do not need to call `ray.get`** inside of these remote functions.

In [3]:
@ray.remote
def load_data(filename):
    time.sleep(0.1)
    return np.ones((1000, 100))

@ray.remote
def normalize_data(data):
    time.sleep(0.1)
    return data - np.mean(data, axis=0)

@ray.remote
def extract_features(normalized_data):
    time.sleep(0.1)
    return np.hstack([normalized_data, normalized_data ** 2])

@ray.remote
def compute_loss(features):
    num_data, dim = features.shape
    time.sleep(0.1)
    return np.sum((np.dot(features, np.ones(dim)) - np.ones(num_data)) ** 2)

assert hasattr(load_data, 'remote'), 'load_data must be a remote function'
assert hasattr(normalize_data, 'remote'), 'normalize_data must be a remote function'
assert hasattr(extract_features, 'remote'), 'extract_features must be a remote function'
assert hasattr(compute_loss, 'remote'), 'compute_loss must be a remote function'

**EXERCISE:** The loop below takes too long. Parallelize the four passes through the loop by turning `load_data`, `normalize_data`, `extract_features`, and `compute_loss` into remote functions and then retrieving the losses with `ray.get`.

**NOTE:** You should only use **ONE** call to `ray.get`. For example, the object ID returned by `load_data` should be passed directly into `normalize_data` without needing to be retrieved by the driver.

In [5]:
# Sleep a little to improve the accuracy of the timing measurements below.
time.sleep(2.0)
start_time = time.time()

losses = []
for filename in ['file1', 'file2', 'file3', 'file4']:
    inner_start = time.time()

    data = load_data.remote(filename)
    normalized_data = normalize_data.remote(data)
    features = extract_features.remote(normalized_data)
    loss = compute_loss.remote(features)
    losses.append(loss)
    
    inner_end = time.time()
    
    if inner_end - inner_start >= 0.1:
        raise Exception('You may be calling ray.get inside of the for loop! '
                        'Doing this will prevent parallelism from being exposed. '
                        'Make sure to only call ray.get once outside of the for loop.')

print('The losses are {}.'.format(losses) + '\n')
loss = sum(ray.get(losses))

end_time = time.time()
duration = end_time - start_time

print('The loss is {}. This took {} seconds. Run the next cell to see '
      'if the exercise was done correctly.'.format(loss, duration))

The losses are [ObjectID(c2a79d56f2feffffffff0100000000c001000000), ObjectID(f0f55112a837ffffffff0100000000c001000000), ObjectID(15835de56970ffffffff0100000000c001000000), ObjectID(604166a15dd4ffffffff0100000000c001000000)].

The loss is 4000.0. This took 0.7030258178710938 seconds. Run the next cell to see if the exercise was done correctly.


**VERIFY:** Run some checks to verify that the changes you made to the code were correct. Some of the checks should fail when you initially run the cells. After completing the exercises, the checks should pass.

In [6]:
assert loss == 4000
assert duration < 0.8, ('The loop took {} seconds. This is too slow.'
                        .format(duration))
assert duration > 0.4, ('The loop took {} seconds. This is too fast.'
                        .format(duration))

print('Success! The example took {} seconds.'.format(duration))

Success! The example took 0.7030258178710938 seconds.


**EXERCISE:** Use the UI to view the task timeline and to verify that the four tasks were executed in parallel. You can do this as follows.

1. Run the following cell to generate a JSON file containing the profiling data.
2. Download the timeline file by right clicking on `timeline02.json` in the navigator to the left and choosing **"Download"**.
3. Open [chrome://tracing/](chrome://tracing/) in the Chrome web browser, click on the **"Load"** button and load the downloaded JSON file.

To navigate within the timeline, do the following.
- Move around by clicking and dragging.
- Zoom in and out by holding **alt** and scrolling.

**NOTE:** The timeline visualization will only work in **Chrome**.

In [7]:
ray.timeline(filename="timeline02.json")

In [1]:
!pip install dask[complete]

     |████████████████████████████████| 55 kB 2.5 MB/s eta 0:00:011
     |████████████████████████████████| 98 kB 4.4 MB/s eta 0:00:011
     |████████████████████████████████| 672 kB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 272 kB 11.7 MB/s eta 0:00:01


In [1]:
from distributed import Client, LocalCluster

client = Client(LocalCluster(n_workers=4, processes=True))

In [2]:
import dask
import numpy as np
import time

@dask.delayed
def load_data(filename):
    time.sleep(0.1)
    return np.ones((1000, 100))

@dask.delayed
def normalize_data(data):
    time.sleep(0.1)
    return data - np.mean(data, axis=0)

@dask.delayed
def extract_features(normalized_data):
    time.sleep(0.1)
    return np.hstack([normalized_data, normalized_data ** 2])

@dask.delayed
def compute_loss(features):
    num_data, dim = features.shape
    time.sleep(0.1)
    return np.sum((np.dot(features, np.ones(dim)) - np.ones(num_data)) ** 2)

assert hasattr(load_data, 'remote'), 'load_data must be a remote function'
assert hasattr(normalize_data, 'remote'), 'normalize_data must be a remote function'
assert hasattr(extract_features, 'remote'), 'extract_features must be a remote function'
assert hasattr(compute_loss, 'remote'), 'compute_loss must be a remote function'

In [4]:
start_time = time.time()

losses = []
for filename in ['file1', 'file2', 'file3', 'file4']:
    inner_start = time.time()

    data = load_data(filename)
    normalized_data = normalize_data(data)
    features = extract_features(normalized_data)
    loss = compute_loss(features)
    losses.append(loss)
    
    inner_end = time.time()
    
    if inner_end - inner_start >= 0.1:
        raise Exception('You may be calling ray.get inside of the for loop! '
                        'Doing this will prevent parallelism from being exposed. '
                        'Make sure to only call ray.get once outside of the for loop.')

print('The losses are {}.'.format(losses) + '\n')
loss = dask.compute(sum(losses))[0]

end_time = time.time()
duration = end_time - start_time

print('The loss is {}. This took {} seconds. Run the next cell to see '
      'if the exercise was done correctly.'.format(loss, duration))

The losses are [Delayed('compute_loss-f0ea7db4-5b79-420b-ae10-e359f730924e'), Delayed('compute_loss-88162b97-9f6c-4953-b1b5-be04c9eb0871'), Delayed('compute_loss-a4e28890-ee11-4d76-8d61-629f2fbdf7f0'), Delayed('compute_loss-b006a3a1-2251-46b7-9c15-2d3ddde64849')].

The loss is 4000.0. This took 0.574378252029419 seconds. Run the next cell to see if the exercise was done correctly.


In [5]:
assert loss == 4000
assert duration < 0.8, ('The loop took {} seconds. This is too slow.'
                        .format(duration))
assert duration > 0.4, ('The loop took {} seconds. This is too fast.'
                        .format(duration))

print('Success! The example took {} seconds.'.format(duration))

Success! The example took 0.574378252029419 seconds.


### Application: Parallel web-scraping

One useful application of what we have learned so far is to scrape information from the web. We will illustrate this in a toy setting, but the same principles apply on a large scale where crawling through websites, parsing them and extracting useful content (e.g. for building a search index or populating a database) is often very computationally demanding.

We break up the process into multiple steps. We first grab the raw HTML of the website using Python's requests package. Then, we use BeautifulSoup to parse the HTML to find the relevant information. Finally, we populate a pandas DataFrames so that we are able to work with the data.

To demonstrate this, we scrape GitHub commits to see the latest commits on several repositories.

In [1]:
!pip install requests
!pip install bs4

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import ray
import requests
import time

ray.init(num_cpus=4, ignore_reinit_error=True)

2021-02-01 17:50:29,049	INFO resource_spec.py:205 -- Starting Ray with 1.03 GiB memory available for workers and up to 0.51 GiB for objects. You can adjust these settings with ray.remote(memory=<bytes>, object_store_memory=<bytes>).
2021-02-01 17:50:29,344	WARNING services.py:1237 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-size' to 'docker run'.


{'node_ip_address': '10.0.10.120',
 'redis_address': '10.0.10.120:26784',
 'object_store_address': '/tmp/ray/session_2021-02-01_17-50-29_048543_1890/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-01_17-50-29_048543_1890/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2021-02-01_17-50-29_048543_1890'}

The following function uses these libraries to parse the latest commits from several repositories on GitHub.

In [2]:
@ray.remote
def fetch_commits(repo):
    url = 'https://github.com/{}/commits/master'.format(repo)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    results = []
    for commit_elt in soup.find_all('li', class_='commit'):
        title = commit_elt.find_all('a', class_='message')[0].attrs.get('aria-label').split('\n')[0]
        link_elts = commit_elt.find_all('a', class_='issue-link')
        link = None
        for le in link_elts:
            if 'issue' in le.attrs['href'].lower():
                link = le.attrs['href']
        results.append(dict(title=title, link=link))

    df = pd.DataFrame(results)
    df['repository'] = repo
    return df

Let's try this out to get results for some ray related topics serially.

In [3]:
start = time.time()
repos = ["ray-project/ray", "modin-project/modin", "tensorflow/tensorflow", "apache/arrow"]
results = []
for repo in repos:
    df = fetch_commits.remote(repo)
    results.append(df)

results = ray.get(results)
df = pd.concat(results, sort=False)
duration = time.time() - start
print("Constructing the dataframe took {} seconds.".format(duration))

Constructing the dataframe took 10.213820934295654 seconds.


**EXERCISE**: Speed up the above serial query by making `fetch_commits` a remote function in order to scrape GitHub results in parallel. Then, see a sample of the data scraped below and feel free to play with the data to find other resources to learn more about these libraries!

In [4]:
df

,repository


In [1]:
from bs4 import BeautifulSoup
import dask
import pandas as pd
import requests
import time

@dask.delayed
def fetch_commits(repo):
    url = 'https://github.com/{}/commits/master'.format(repo)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    results = []
    for commit_elt in soup.find_all('li', class_='commit'):
        title = commit_elt.find_all('a', class_='message')[0].attrs.get('aria-label').split('\n')[0]
        link_elts = commit_elt.find_all('a', class_='issue-link')
        link = None
        for le in link_elts:
            if 'issue' in le.attrs['href'].lower():
                link = le.attrs['href']
        results.append(dict(title=title, link=link))

    df = pd.DataFrame(results)
    df['repository'] = repo
    return df

In [2]:
start = time.time()
repos = ["ray-project/ray", "modin-project/modin", "tensorflow/tensorflow", "apache/arrow"]
results = []
for repo in repos:
    df = fetch_commits(repo)
    results.append(df)

results = dask.compute(*results)
df = pd.concat(results, sort=False)
duration = time.time() - start
print("Constructing the dataframe took {} seconds.".format(duration))

Constructing the dataframe took 4.518927574157715 seconds.


In [3]:
df

,repository
